## Blinded data tests

In [ ]:
import os
import sys
import itertools
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import lsstypes as types

sys.path.append('/global/cfs/cdirs/desicollab/users/epaillas/code/desiblind')
from desiblind import TracerPowerSpectrumMultipolesBlinder

In [22]:
def load_mesh2_spectrum_poles(filename):
    result = types.read(filename)
    k = result.get(ells=0).coords('k')
    Pk = {ell: result.get(ells=ell).values() for ell in result.ells}
    return k, Pk

In [ ]:
def get_namespace(tracer, zrange):
    return {
        ('BGS_BRIGHT-21.35', (0.1, 0.4)): 'BGS_z0',
        ('LRG', (0.4, 0.6)): 'LRG_z0',
        ('LRG', (0.6, 0.8)): 'LRG_z1',
        ('LRG', (0.8, 1.1)): 'LRG_z2',
        ('ELG_LOPnotqso', (0.8, 1.1)): 'ELG_z0',
        ('ELG_LOPnotqso', (1.1, 1.6)): 'ELG_z1',
        ('QSO', (0.8, 2.1)): 'QSO_z0',
    }[(tracer, zrange)]

def get_blinded(spectrum, tracer='LRG', zrange=(0.4 , 0.6)):
    from desiblind import TracerPowerSpectrumMultipolesBlinder
    blinded_label = get_namespace(tracer, zrange)
    blinded_spectrum = TracerPowerSpectrumMultipolesBlinder.apply_blinding(
        name=blinded_label,
        data=spectrum,
    )
    return blinded_spectrum

def get_synthetic_data(statistic='mesh2_spectrum_poles', tracer='LRG', zrange=(0.4, 0.6),
    region='GCcomb', ells=[0, 2, 4], weights='default_fkp', klim=(0., 0.3), rebin=5):
    """
    Synthetic data from Abacus-HF mocks for testing.
    """    
    dirname = Path('/pscratch/sd/s/shengyu/Y3/blinded/dr1-v1.5/')

    zmin, zmax = zrange
    covariance = types.read(dirname / f'covariance_{statistic}_{tracer}_z{zmin}-{zmax}_{region}_{weights}.h5')
    observable = covariance.observable.select(k=slice(0, None, rebin)).select(k=klim).get(ells=ells)

    covariance = covariance.at.observable.match(observable)

    window = types.read(dirname / f'window_{statistic}_{tracer}_z{zmin}-{zmax}_{region}_{weights}.h5')
    window = window.at.observable.match(observable)
    window = window.at.theory.select(k=(0, 0.35))

    return observable, covariance, window

def get_measurement_fn(kind='mesh2_spectrum_poles', version='dr1-v1.5', recon=None, tracer='LRG', region='NGC', zrange=(0.8, 1.1), cut=None, auw=None, nran = 18, weight_type='default', **kwargs):
    # base_dir = Path(f'/global/cfs/projectdirs/desi/mocks/cai/mock-challenge-cutsky-dr2/')
    # base_dir = base_dir / (f'blinded_{recon}' if recon else 'blinded')
    # base_dir = Path(f'/global/cfs/projectdirs/desi/mocks/cai/mock-challenge-cutsky-dr2/blinded_data/{version}/data_splits')
    base_dir = Path(f'/pscratch/sd/s/shengyu/Y3/blinded/{version}/data_splits')
    if cut: cut = '_thetacut'
    else: cut = ''
    if auw: auw = '_auw'
    else: auw = ''
    return str(base_dir / f'{kind}_{tracer}_z{zrange[0]:.1f}-{zrange[1]:.1f}_{region}_{weight_type}{auw}{cut}_nran{nran}.h5')


In [12]:
fn = '/global/cfs/projectdirs/desi/mocks/cai/mock-challenge-cutsky-dr2/blinded_data/dr2-v2/data_splits/blinded_mesh2_spectrum_poles_LRG_z0.6-0.8_NGC_default_nran18.h5'
spectrum = types.read(fn)
spectrum

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/global/cfs/projectdirs/desi/mocks/cai/mock-challenge-cutsky-dr2/blinded_data/dr2-v2/data_splits/blinded_mesh2_spectrum_poles_LRG_z0.6-0.8_NGC_default_nran18.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# default_bin = [
# ('BGS', (0.1, 0.4)),
# ('LRG', (0.4, 0.6)),
# ('LRG', (0.6, 0.8)),
# ('LRG', (0.8, 1.1)),
# ('ELG_LOPnotqso', (0.8, 1.1)),
# ('ELG_LOPnotqso', (1.1, 1.6)),
# ('QSO', (0.8, 2.1))
# ]
# regions = ['N']

# version = 'dr1-v1.5'
# for (tracer, zrange), region in itertools.product(default_bin, regions):
#     if 'BGS' in tracer:
#             tracer = 'BGS_BRIGHT-21.5' if 'dr1' in version else 'BGS_BRIGHT-21.35'
#     catalog_args = dict(version=version, region=region, tracer=tracer, zrange=zrange, weight_type='default_fkp', nran=18)
#     fn = get_measurement_fn(**catalog_args)
#     spectrum = types.read(fn)
#     blineded_fn =  get_measurement_fn(**catalog_args, kind= 'blinded_mesh2_spectrum_poles')
#     blineded_spectrum = get_blinded(spectrum, tracer=tracer, zrange=zrange)
#     blineded_spectrum.write(blineded_fn)
#     os.remove(fn)